<a href="https://colab.research.google.com/github/DhruvaBansal00/ConfidentMT/blob/master/BoostedBinaryClassifers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
!ls
%cd drive/My Drive/ConfidentMachineTranslation/flores
!ls
!pwd

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
drive  sample_data
/content/drive/My Drive/ConfidentMachineTranslation/flores
analysis		       data		       LM_Thresholding.ipynb
Analysis		       data-bin		       NCD_Analysis.ipynb
backward_models		       Ensembles	       noisychannel
BoostedBinaryClassifers.ipynb  Ensembling	       NoisyChannel.ipynb
checkpoints		       FLORES.ipynb	       Resources
ClassificationDataset	       LanguageAnalysis.ipynb  scripts
configs			       language_models
/content/drive/

In [1]:
# prints how much GPU RAM is available
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

  Created wheel for gputil: filename=GPUtil-1.4.0-cp36-none-any.whl size=7413 sha256=11a6390ea47ac81e8d42a0e29cbf3e6f2703217c9a8c091e6bd9b8eccf2fca4d
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil
Gen RAM Free: 12.8 GB  | Proc size: 159.4 MB
GPU RAM Free: 16280MB | Used: 0MB | Util   0% | Total 16280MB


In [43]:
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from itertools import zip_longest

def printDatasetClassProp(Y): 
    classes = {}
    total = len(Y)
    for i in Y:
        if i in classes:
            classes[i] += 1
        else:
            classes[i] = 1
    
    for cls in classes:
        print("Proportion in class " + str(cls) + " = " + str(classes[cls]/total))

def datasetReader(featureFile, labelFile):
    files = [featureFile, labelFile]

    X = []
    Y = []

    for lines in zip_longest(*files, fillvalue=''):
        currX, currY = lines[0], float(lines[1].strip("\n"))
        Xarr = []
        features = currX.split()
        for feature in features:
            Xarr.append(float(feature.strip(",").strip("\n")))
        X.append(Xarr)
        Y.append(currY)
    
    return np.array(X), np.array(Y)


def trainRandomForestClassifier(X, Y, maxDepth):
    print("Training Random Forrest Classifier")
    clf = RandomForestClassifier(max_depth=maxDepth, random_state=42)
    clf.fit(X, Y)
    return clf

def calculateAccuracy(predictedClasses, groundTruth):
    correct_accepted = 0
    total_accepted = 0

    correct_rejected = 0
    total_rejected = 0

    for i in range(len(predictedClasses)):
        if groundTruth[i] == 1:
            total_accepted += 1
            if predictedClasses[i] == groundTruth[i]:
                correct_accepted += 1
        else:
            total_rejected += 1
            if predictedClasses[i] == groundTruth[i]:
                correct_rejected += 1


    print("Correctly accepted = " + str(correct_accepted/total_accepted))
    print("Incorrectly accepted = " + str(1 - correct_rejected/total_rejected))
    print("Correctly rejected = " + str(correct_rejected/total_rejected))
    print("Incorrectly rejected = " + str(1 - correct_accepted/total_accepted))

    print("Total Accuracy = " + str((correct_accepted + correct_rejected)/(total_accepted + total_rejected)))
trainset = "valid"
testset = "test"

trainFeatures = open("ClassificationDataset/"+trainset+"/features.txt")
testFeatures = open("ClassificationDataset/"+testset+"/features.txt")

trainLabels = open("ClassificationDataset/"+trainset+"/result.txt")
testLabels = open("ClassificationDataset/"+testset+"/result.txt")

trainX, trainY = datasetReader(trainFeatures, trainLabels)
testX, testY = datasetReader(testFeatures, testLabels)

printDatasetClassProp(trainY)



rndForest = trainRandomForestClassifier(trainX, trainY, 8)
predictions = np.array(rndForest.predict(testX))
calculateAccuracy(predictions, testY)


Proportion in class 0.0 = 0.8014849550605705
Proportion in class 1.0 = 0.19851504493942945
Training Random Forrest Classifier
Correctly accepted = 0.24200278164116829
Incorrectly accepted = 0.03308128544423439
Correctly rejected = 0.9669187145557656
Incorrectly rejected = 0.7579972183588317
Total Accuracy = 0.783068783068783
